In [31]:
!pip install together pandas numpy matplotlib seaborn python-docx PyMuPDF pdfplumber openpyxl pillow streamlit


In [32]:
!pip install streamlit pandas matplotlib seaborn PyMuPDF python-docx together



In [33]:
code = """
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fitz  # PyMuPDF
import docx
import together

together.api_key = "f0110cd1548f9e5a6203b7c1aefa832218c5c3ae326230fc5de934dadf4bcfbc"
LLAMA_MODEL = "meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8"

def extract_text_from_file(file_path):
    ext = file_path.split('.')[-1].lower()
    if ext == 'txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif ext == 'pdf':
        with fitz.open(file_path) as doc:
            return "\\n".join([page.get_text() for page in doc])
    elif ext == 'docx':
        return "\\n".join([para.text for para in docx.Document(file_path).paragraphs])
    elif ext in ['png', 'jpg', 'jpeg']:
        return "[Image uploaded. OCR or image analysis not implemented.]"
    elif ext in ['csv', 'xlsx']:
        if ext == 'csv':
            df = pd.read_csv(file_path)
        else:
            df = pd.read_excel(file_path)
        return df
    else:
        return "Unsupported file type."

def ask_llm(prompt, system_prompt="You are a data analyst agent."):
    response = together.Complete.create(
        prompt=f"{system_prompt}\\nUser: {prompt}\\nAssistant:",
        model=LLAMA_MODEL,
        max_tokens=512,
        temperature=0.7,
        top_p=0.9,
        stop=["User:"]
    )
    if 'choices' in response and len(response['choices']) > 0:
        return response['choices'][0]['text'].strip()
    else:
        return "Error: No response from the model."

def analyze_dataframe(df):
    summary = df.describe(include='all').to_string()
    info = df.dtypes.to_string()
    return f"Data Types:\\n{info}\\n\\nSummary Statistics:\\n{summary}"

def create_visualization(df, x=None, y=None, chart_type='bar'):
    plt.figure(figsize=(10,6))
    if chart_type == 'bar' and x and y:
        sns.barplot(x=df[x], y=df[y])
    elif chart_type == 'hist' and x:
        sns.histplot(df[x])
    elif chart_type == 'scatter' and x and y:
        sns.scatterplot(x=df[x], y=df[y])
    elif chart_type == 'line' and x and y:
        sns.lineplot(x=df[x], y=df[y])
    else:
        print("Unsupported chart type or missing variables.")
        return
    plt.title(f"{chart_type.capitalize()} Plot")
    plt.tight_layout()
    plt.show()
"""
with open("data_analyst_agent.py", "w") as f:
    f.write(code)


In [34]:
app_code = '''
import streamlit as st
import pandas as pd
from data_analyst_agent import extract_text_from_file, ask_llm, analyze_dataframe, create_visualization

st.title("📊 Data Analyst Agent (Llama-4 Maverick)")

uploaded = st.file_uploader("Upload your file", type=["csv", "xlsx", "pdf", "txt", "docx", "png", "jpg"])
if uploaded:
    with open(uploaded.name, "wb") as f:
        f.write(uploaded.read())
    parsed = extract_text_from_file(uploaded.name)

    if isinstance(parsed, pd.DataFrame):
        st.write(parsed.head())
        st.text(analyze_dataframe(parsed))

        q = st.text_input("Ask a question about the data")
        if q:
            summary = parsed.head(10).to_csv(index=False)
            res = ask_llm(f"Dataset sample:\\n{summary}\\n\\nQuestion: {q}")
            st.markdown("**Answer:** " + res)
    else:
        st.text_area("Extracted Text", parsed[:2000] if isinstance(parsed, str) else str(parsed))
        q = st.text_input("Ask something about the document")
        if q:
            res = ask_llm(f"Text:\\n{parsed[:2000]}\\n\\nQuestion: {q}")
            st.markdown("**Answer:** " + res)
'''
with open("app.py", "w") as f:
    f.write(app_code)


In [35]:
!pip install -q streamlit pyngrok pandas matplotlib seaborn python-docx PyMuPDF openpyxl together Pillow

In [36]:
!ngrok config add-authtoken 2yV3LXNaGTor4BJHpfIfT8uM7A2_7nNXemei7fvzecvztga3g


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [37]:
from PIL import Image


In [39]:
from pyngrok import ngrok

# Kill any existing tunnels
ngrok.kill()

# Start a new tunnel on port 8501
public_url = ngrok.connect(8501)
print("🌍 Streamlit app running at:", public_url)

# Launch Streamlit app
!streamlit run app.py &> /dev/null &


🌍 Streamlit app running at: NgrokTunnel: "https://ea78-34-83-217-48.ngrok-free.app" -> "http://localhost:8501"
